In [12]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [58]:

import numpy as np 

import pandas as pd 

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
# Linki do plików train.csv i test.csv
url='https://raw.githubusercontent.com/Adamek548/Titanic/main/test.csv'
urlsecond='https://raw.githubusercontent.com/Adamek548/Titanic/main/train.csv'

#    Pobieranie danych ze strony

test_dow = pd.read_csv(url)
train_dow = pd.read_csv(urlsecond)

#    Analiza danych

train_dow.info()
test_dow.info()

# Wykresy słupkowe 

# def bar_chart(feature):
#     survived = train_dow[train_dow['Survived']==1][feature].value_counts()
#     dead = train_dow[train_dow['Survived']==0][feature].value_counts()
#     dow = pd.DataFrame([survived,dead])
#     dow.index = ['Survived','Dead']
#     dow.plot(kind='bar',stacked=True, figsize=(10,5))

# bar_chart('Sex')
# bar_chart('Pclass')
# bar_chart('Embarked')
# bar_chart('Parch')
# bar_chart('SibSp')

# Procentowy brak danych w rożnych kategoriach

total = train_dow.isnull (). sum (). sort_values(ascending = False ) 
percent_1 = train_dow.isnull (). sum () / train_dow.isnull (). count () * 100 
percent_2 = (round (percent_1, 1) ) .sort_values(ascending = False ) 
missing_data = pd.concat ([total, percent_2], axis = 1, keys = ['Łącznie', '%']) 
missing_data.head (5)

#   Usunięcie kolumny 'PASSENGERID' z pliku train.csv

train_dow = train_dow.drop(['PassengerId'], axis=1)

#   Obrabianie danych dotyczących kabiny

import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}   # Oznaczenie pokładu
data = [train_dow, test_dow]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
train_dow = train_dow.drop(['Cabin'], axis=1)
test_dow = test_dow.drop(['Cabin'], axis=1)

#   Dane dotyczące wieku-obróbka

data = [train_dow, test_dow]

for dataset in data:
    mean = train_dow["Age"].mean()
    std = test_dow["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # obliczenie losowej liczby między średnią std i is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # wypełnienie wartości NaN(not a number) w kolumnie "Age" wygenerowanymi losowo wartościami
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_dow["Age"].astype(int)
train_dow["Age"].isnull().sum()



train_dow['Embarked'].describe()


#  Uzupełnienie w kolumnie 'Embarked'

common_value = 'S'
data = [train_dow, test_dow]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

#   Konwersja typu danych

data = [train_dow, test_dow]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)



data = [train_dow, test_dow]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # wyodrębnienie tytułów
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # zamiana tytułów na bardziej popularne
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # zamiana tytułów na liczby
    dataset['Title'] = dataset['Title'].map(titles)
    # wypełnienie rubryk z NaN(not a number) zerami
    dataset['Title'] = dataset['Title'].fillna(0)
train_dow = train_dow.drop(['Name'], axis=1)
test_dow = test_dow.drop(['Name'], axis=1)

#  Zamiana płci na wartosci zer i jedynek

genders = {"male": 0, "female": 1}
data = [train_dow, test_dow]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

# Ucuniecie kolumny 'Ticket'

train_dow = train_dow.drop(['Ticket'], axis=1)
test_dow = test_dow.drop(['Ticket'], axis=1)

# Zamiana portow na numery

ports = {"S": 0, "C": 1, "Q": 2}
data = [train_dow, test_dow]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

#   Utworzenie grup wiekowych

data = [train_dow, test_dow]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 23), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 23) & (dataset['Age'] <= 28), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 28) & (dataset['Age'] <= 34), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 34) & (dataset['Age'] <= 45), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 45) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6


# Utworzenie kategorii w kategorii Fare

data = [train_dow, test_dow]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.451), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.451) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)

# Dodano WIEK => KLASA

data = [train_dow, test_dow]
for dataset in data:
    dataset['Age_Class']= dataset['Age']* dataset['Pclass']

# Model uczacy random_forest
X_train = train_dow.drop ("Survived", axis = 1) 
Y_train = train_dow ["Survived"] 
X_test = test_dow.drop ("PassengerId", axis = 1) .copy ()


random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

from sklearn.model_selection import cross_val_score
las = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(las, X_train, Y_train, cv=10, scoring = "accuracy")

print("\n")
print(40 * "-")
print("\n")
print("Wyniki drzewa:", scores)
print("Odchylenie standardowe:", scores.std())
print("Zaokrąglony wynik sprawnosci drzewa decyzyjnego:")
round(np.mean(scores)*100, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

82.72